In [16]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, make_scorer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler # For SVM
import seaborn as sns # Keep if you use it for plotting later
from alpha_vantage.timeseries import TimeSeries # For data fetching

from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Get the API key
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

# Define target labels for multi-class classification
# These labels correspond to your -1 (Sell), 0 (Hold), 1 (Buy) strategy
LABELS = [-1, 0, 1]

# Create a custom multi-class ROC AUC scorer for GridSearchCV
# Using 'ovr' (One-vs-Rest) strategy
multi_class_roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr', labels=LABELS)

In [17]:
# Cell 2: Fetch Data from Alpha Vantage
# Using Apple (AAPL) as an example ticker. You can change this.
ts = TimeSeries(key=API_KEY, output_format='pandas')
data, meta_data = ts.get_daily(symbol='AAPL', outputsize='full')


# Clean and rename columns for consistency
data.columns = ['open', 'high', 'low', 'close', 'volume']
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)

# Use 'close' for all calculations as per your original notebook's intent,
# unless 'adjusted_close' is explicitly preferred for adjusted prices.
# If you want to use adjusted_close, replace 'close' with 'adjusted_close' everywhere.

print("✅ Data fetched and prepared.")
print(data.head())

✅ Data fetched and prepared.
             open   high    low  close     volume
date                                             
1999-11-01  80.00  80.69  77.37  77.62  2487300.0
1999-11-02  78.00  81.69  77.31  80.25  3564600.0
1999-11-03  81.62  83.25  81.00  81.50  2932700.0
1999-11-04  82.06  85.37  80.62  83.62  3384700.0
1999-11-05  84.62  88.37  84.00  88.31  3721500.0


In [18]:
# Cell 3: Feature Engineering
# Create a copy for feature engineering
feature_data = data.copy()

# Calculate Daily Returns for feature
feature_data['daily_return'] = feature_data['close'].pct_change()

# Calculate RSI
rsi_indicator = RSIIndicator(close=feature_data['close'], window=14)
feature_data['rsi'] = rsi_indicator.rsi()

# Calculate MACD
macd_indicator = MACD(close=feature_data['close'])
feature_data['macd'] = macd_indicator.macd()
feature_data['macd_signal'] = macd_indicator.macd_signal()

# Drop rows with NaN values resulting from indicator calculations
feature_data.dropna(inplace=True)

print("✅ Features engineered.")
print(feature_data.head())

✅ Features engineered.
              open    high     low   close     volume  daily_return  \
date                                                                  
1999-12-17  100.87  102.00   98.50  100.00  4419700.0      0.017191   
1999-12-20   99.56   99.62   96.62   98.00  2535600.0     -0.020000   
1999-12-21   98.19  103.06   97.94  102.50  2746400.0      0.045918   
1999-12-22  102.87  104.56   98.75   99.94  2920300.0     -0.024976   
1999-12-23  101.81  104.25  101.06  103.50  2049400.0      0.035621   

                  rsi      macd  macd_signal  
date                                          
1999-12-17  52.729509  2.724819     4.846229  
1999-12-20  49.896383  2.279161     4.332816  
1999-12-21  55.667989  2.263000     3.918852  
1999-12-22  51.998303  2.020332     3.539148  
1999-12-23  56.311373  2.091173     3.249553  


In [19]:
# Cell 4: Define Multi-Class Target Variable (-1 for Sell, 0 for Hold, 1 for Buy)

# Define return for the next period (shifted by -1 to predict the next day)
# Note: We are using 'close' price for this return calculation.
# Ensure this is consistent with your trading logic.
feature_data['future_return'] = feature_data['close'].pct_change().shift(-1)

# Define your thresholds for Buy/Sell signals
# These thresholds can be tuned based on desired sensitivity
# For example, a 0.1% move up or down might trigger a signal, otherwise it's a hold.
BUY_THRESHOLD = 0.001   # e.g., predict a buy if next day's return is > 0.1%
SELL_THRESHOLD = -0.001 # e.g., predict a sell if next day's return is < -0.1%

# Initialize target column with '0' (Hold/Neutral)
feature_data['target'] = 0

# Assign '1' (Buy) signals
feature_data.loc[feature_data['future_return'] > BUY_THRESHOLD, 'target'] = 1

# Assign '-1' (Sell) signals
feature_data.loc[feature_data['future_return'] < SELL_THRESHOLD, 'target'] = -1

# Drop the 'future_return' column to prevent look-ahead bias
feature_data.drop(columns=['future_return'], inplace=True)

# Drop any remaining rows with NaN values (e.g., last row after target shift)
feature_data.dropna(inplace=True)

print("✅ Multi-class target variable defined.")
print(feature_data['target'].value_counts()) # Check distribution of classes
print(feature_data.head())

✅ Multi-class target variable defined.
target
 1    3186
-1    2886
 0     346
Name: count, dtype: int64
              open    high     low   close     volume  daily_return  \
date                                                                  
1999-12-17  100.87  102.00   98.50  100.00  4419700.0      0.017191   
1999-12-20   99.56   99.62   96.62   98.00  2535600.0     -0.020000   
1999-12-21   98.19  103.06   97.94  102.50  2746400.0      0.045918   
1999-12-22  102.87  104.56   98.75   99.94  2920300.0     -0.024976   
1999-12-23  101.81  104.25  101.06  103.50  2049400.0      0.035621   

                  rsi      macd  macd_signal  target  
date                                                  
1999-12-17  52.729509  2.724819     4.846229      -1  
1999-12-20  49.896383  2.279161     4.332816       1  
1999-12-21  55.667989  2.263000     3.918852      -1  
1999-12-22  51.998303  2.020332     3.539148       1  
1999-12-23  56.311373  2.091173     3.249553      -1  


In [20]:
# Cell 5: Prepare Data for Modeling

# Define features (X) and target (y)
# Exclude original price columns and other non-feature columns
features = ['daily_return', 'rsi', 'macd', 'macd_signal'] # These are the engineered features
X = feature_data[features]
y = feature_data['target']

# Split data into training and testing sets
# shuffle=False is crucial for time series data to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

# Initialize StandardScaler for SVM. Random Forest and XGBoost are generally less sensitive to scaling.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled arrays back to DataFrames with original indices and columns
X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

print("✅ Data prepared for modeling.")
print(f"Train set shape: {X_train.shape}, Test set shape: {X_test.shape}")

✅ Data prepared for modeling.
Train set shape: (5134, 4), Test set shape: (1284, 4)


In [21]:
# Cell 6: Train and Evaluate RandomForestClassifier

print("--- RandomForestClassifier ---")
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced') # Adding class_weight for imbalance
rf_model.fit(X_train, y_train) # Using unscaled data for RF

y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test) # Get probabilities for all classes

print("\n📈 Classification Report (RandomForest):")
print(classification_report(y_test, y_pred_rf, labels=LABELS, zero_division=0)) # zero_division=0 to suppress warnings

print("ROC AUC Score (RandomForest):", roc_auc_score(y_test, y_proba_rf, multi_class='ovr', labels=LABELS))

--- RandomForestClassifier ---

📈 Classification Report (RandomForest):
              precision    recall  f1-score   support

          -1       0.47      0.46      0.46       561
           0       0.25      0.01      0.03        76
           1       0.52      0.57      0.54       647

    accuracy                           0.49      1284
   macro avg       0.41      0.35      0.34      1284
weighted avg       0.48      0.49      0.48      1284

ROC AUC Score (RandomForest): 0.5319074390227815


In [22]:
# Cell 7: Hyperparameter Tuning for RandomForestClassifier (GridSearchCV)

print("--- Hyperparameter Tuning (RandomForest with GridSearchCV) ---")

# 1. Definimos hiperparámetros a explorar
# Reducción de opciones para una ejecución más rápida
param_grid = {
    'n_estimators': [50, 100],        # Reduced from [50, 100, 200]
    'max_depth': [5, 10],             # Reduced from [3, 5, 10, None]
    'min_samples_split': [5, 10],     # Reduced from [2, 5, 10]
    'min_samples_leaf': [2, 4],       # Reduced from [1, 2, 4]
    'class_weight': ['balanced']
}
# New total combinations: 2 * 2 * 2 * 2 * 1 = 16 combinations
# With cv=3: 16 * 3 = 48 fits (significantly faster than 540)

# Create a custom multi-class ROC AUC scorer for GridSearchCV
from sklearn.metrics import make_scorer
# Define your unique labels for the target variable (same as in Cell 1)
LABELS = [-1, 0, 1]
multi_class_roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr', labels=LABELS)


# 2. Instanciamos el clasificador base
from sklearn.ensemble import RandomForestClassifier
rf_base = RandomForestClassifier(random_state=42)

# 3. Creamos el GridSearch
grid_search_rf = GridSearchCV(estimator=rf_base,
                              param_grid=param_grid,
                              cv=3,  # Reduced cross-validation folds for speed
                              scoring=multi_class_roc_auc_scorer, # Use the custom multi-class scorer
                              verbose=2,
                              n_jobs=-1)

# 4. Entrenamos
grid_search_rf.fit(X_train, y_train)

# 5. Mostramos la mejor combinación
print("🧠 Best Parameters (RandomForest):")
print(grid_search_rf.best_params_)

# 6. Usamos el mejor modelo
best_rf_model = grid_search_rf.best_estimator_

# 7. Evaluamos
y_pred_best_rf = best_rf_model.predict(X_test)
y_proba_best_rf = best_rf_model.predict_proba(X_test)

print("\n📈 Classification Report (Best RandomForest Model):")
print(classification_report(y_test, y_pred_best_rf, labels=LABELS, zero_division=0))

print(f"ROC AUC (Best RandomForest Model): {roc_auc_score(y_test, y_proba_best_rf, multi_class='ovr', labels=LABELS):.4f}")

--- Hyperparameter Tuning (RandomForest with GridSearchCV) ---
Fitting 3 folds for each of 16 candidates, totalling 48 fits


/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 308, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 408, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
      

[CV] END class_weight=balanced, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END class_weight=balanced, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END class_weight=balanced, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.4s


/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 308, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 408, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
      

[CV] END class_weight=balanced, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.8s


...

[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   1.0s


/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 308, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 408, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
      

[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.9s


/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 308, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/backtesting/my_env/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 408, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
      

[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.8s
🧠 Best Parameters (RandomForest):
{'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}

📈 Classification Report (Best RandomForest Model):
              precision    recall  f1-score   support

          -1       0.47      0.38      0.42       561
           0       0.06      0.30      0.10        76
           1       0.50      0.36      0.42       647

    accuracy                           0.36      1284
   macro avg       0.34      0.35      0.31      1284
weighted avg       0.46      0.36      0.40      1284

ROC AUC (Best RandomForest Model): 0.5285


In [23]:
# Cell 8: Train and Evaluate XGBClassifier

from sklearn.preprocessing import LabelEncoder

print("\n--- XGBClassifier ---")

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform y_train and y_test to non-negative integers
# XGBoost expects classes to be 0, 1, 2, ... for multi-class objectives
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Verify the mapping (optional, for understanding)
# print("Original labels:", le.classes_) # Shows how original labels map to encoded (e.g., -1->0, 0->1, 1->2)

# For XGBoost, the 'objective' needs to be multi-class if target has > 2 classes
# 'multi:softmax' for multi-class classification, returns class labels (encoded)
# 'num_class' specifies the number of unique classes (should be len(LABELS_ENCODED))
xgb_model = XGBClassifier(objective='multi:softmax',
                          num_class=len(le.classes_), # Use length of unique encoded classes
                          use_label_encoder=False,
                          eval_metric='mlogloss',
                          random_state=42)

# Train the model using the encoded labels
xgb_model.fit(X_train, y_train_encoded) # Using unscaled data for XGBoost

# Predict using the trained model (will return encoded labels)
y_pred_xgb_encoded = xgb_model.predict(X_test)

# Inverse transform the predictions back to original labels for evaluation
y_pred_xgb = le.inverse_transform(y_pred_xgb_encoded)

# Get prediction probabilities for all classes (will be ordered by encoded classes 0, 1, 2)
y_proba_xgb = xgb_model.predict_proba(X_test)

print("\n📈 Classification Report (XGBoost):")
# Use original y_test and inverse-transformed y_pred_xgb for the report
print(classification_report(y_test, y_pred_xgb, labels=LABELS, zero_division=0))

# For ROC AUC, pass the original y_test and the probabilities, specifying multi_class and original labels
print("ROC AUC Score (XGBoost):", roc_auc_score(y_test, y_proba_xgb, multi_class='ovr', labels=LABELS))


--- XGBClassifier ---


/workspaces/backtesting/my_env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:52:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📈 Classification Report (XGBoost):
              precision    recall  f1-score   support

          -1       0.44      0.42      0.43       561
           0       0.00      0.00      0.00        76
           1       0.50      0.57      0.53       647

    accuracy                           0.47      1284
   macro avg       0.31      0.33      0.32      1284
weighted avg       0.44      0.47      0.46      1284

ROC AUC Score (XGBoost): 0.5152329662509408


In [24]:
# Cell 9: Train and Evaluate SVM (SVC)

print("\n--- SVM (SVC) ---")
# SVC also supports multi-class classification inherently.
# 'probability=True' is needed if you want predict_proba, which is required for ROC AUC.
# Using scaled data for SVM
svm_model = SVC(probability=True, random_state=42, class_weight='balanced') # Adding class_weight for imbalance
svm_model.fit(X_train_scaled_df, y_train) # Use scaled data

y_pred_svm = svm_model.predict(X_test_scaled_df)
y_proba_svm = svm_model.predict_proba(X_test_scaled_df) # Get probabilities for all classes

print("\n📈 Classification Report (SVM):")
print(classification_report(y_test, y_pred_svm, labels=LABELS, zero_division=0))

print("ROC AUC Score (SVM):", roc_auc_score(y_test, y_proba_svm, multi_class='ovr', labels=LABELS))


--- SVM (SVC) ---

📈 Classification Report (SVM):
              precision    recall  f1-score   support

          -1       0.45      0.35      0.39       561
           0       0.07      0.58      0.13        76
           1       0.51      0.18      0.26       647

    accuracy                           0.28      1284
   macro avg       0.34      0.37      0.26      1284
weighted avg       0.46      0.28      0.31      1284

ROC AUC Score (SVM): 0.5326984107495701


In [25]:
# Cell 10: Classical Signal Generation (SMA Crossover & RSI)

# Important: These traditional signals will generate 0 or 1, not -1.
# If you want them to also generate -1 signals, you'd need to modify their logic.
# For simplicity and to maintain original intent, we keep them binary (Buy/Hold).

# Calculate SMAs on the full feature_data
feature_data['SMA_10'] = feature_data['close'].rolling(window=10).mean()
feature_data['SMA_50'] = feature_data['close'].rolling(window=50).mean()

# SMA Crossover Signal
# 1 if SMA_10 > SMA_50 (Buy), 0 otherwise (Hold)
feature_data['y_pred_crossover'] = np.where(feature_data['SMA_10'] > feature_data['SMA_50'], 1, 0)

# RSI Signal
# 1 if RSI < 30 (Buy - oversold), 0 if RSI > 70 (Hold - overbought), else 0 (Hold)
feature_data['y_pred_rsi'] = np.where(feature_data['rsi'] < 30, 1, 0)
feature_data['y_pred_rsi'] = np.where(feature_data['rsi'] > 70, 0, feature_data['y_pred_rsi']) # Set overbought to 0

# Drop NaNs from SMA calculations
feature_data.dropna(inplace=True)

print("✅ Classical signals generated.")
print(feature_data[['SMA_10', 'SMA_50', 'y_pred_crossover', 'rsi', 'y_pred_rsi']].head())

✅ Classical signals generated.
             SMA_10    SMA_50  y_pred_crossover        rsi  y_pred_rsi
date                                                                  
2000-02-29  114.274  106.1032                 1  54.716535           0
2000-03-01  115.405  106.7094                 1  66.965397           0
2000-03-02  116.193  107.1894                 1  58.014699           0
2000-03-03  117.506  107.6994                 1  61.967428           0
2000-03-06  118.950  108.2144                 1  59.639451           0


In [26]:
# Cell 11: Prepare and Save Signal Data for Backtesting

# All prediction columns should be assigned back to the feature_data DataFrame,
# aligning by index. We need to ensure predictions cover the full time series
# (training and testing periods) for consistent saving.

# Re-initialize prediction columns to fill NaNs for the training period if not already done.
# For simplicity, fill with 0 (no trade) for periods not in X_test.
# This assumes that for backtesting, you either don't trade in training, or use a
# fixed position (like 0) for training period predictions.

# IMPORTANT: Ensure the indices for X_test (and X_train) align with feature_data
# This is handled because X_train/X_test were created from feature_data.

# Assign predictions from models back to feature_data based on their respective test set indices
# For RandomForest (initial model)
feature_data['y_pred'] = 0 # Default to 0 (Hold)
feature_data.loc[X_test.index, 'y_pred'] = y_pred_rf

# For Best RandomForest Model
feature_data['y_pred_best'] = 0
feature_data.loc[X_test.index, 'y_pred_best'] = y_pred_best_rf

# For XGBoost
feature_data['y_pred_xgb'] = 0
feature_data.loc[X_test.index, 'y_pred_xgb'] = y_pred_xgb

# For SVM
feature_data['y_pred_svm'] = 0
feature_data.loc[X_test.index, 'y_pred_svm'] = y_pred_svm


# Select relevant columns for signal_data.csv
# This should include 'close' price and all generated prediction columns
signal_data = feature_data[['close', 'y_pred', 'y_pred_best', 'y_pred_xgb', 'y_pred_svm', 'y_pred_crossover', 'y_pred_rsi']].copy()

# Ensure the index name is 'date' as expected by the backtesting notebook
signal_data.index.name = 'date'

# Define the output path
output_path = '/workspaces/backtesting/investment-portfolio-project/data/signal_data.csv'

# Save the signal data to CSV
signal_data.to_csv(output_path)

print(f"✅ Señales añadidas correctamente. Archivo actualizado: {output_path}")
print(signal_data.tail())

✅ Señales añadidas correctamente. Archivo actualizado: /workspaces/backtesting/investment-portfolio-project/data/signal_data.csv
             close  y_pred  y_pred_best  y_pred_xgb  y_pred_svm  \
date                                                              
2025-06-18  196.58      -1           -1          -1          -1   
2025-06-20  201.00       1           -1           1          -1   
2025-06-23  201.50       1           -1           1           0   
2025-06-24  200.30       1           -1           1          -1   
2025-06-25  201.56       1            0           1           0   

            y_pred_crossover  y_pred_rsi  
date                                      
2025-06-18                 0           0  
2025-06-20                 0           0  
2025-06-23                 0           0  
2025-06-24                 0           0  
2025-06-25                 0           0  


In [27]:
# Cell 12: Save Feature Data (Optional, but good practice)
# You might want to save the feature data for debugging or further analysis later.

feature_data_output_path = '/workspaces/backtesting/investment-portfolio-project/data/feature_data.csv'
feature_data.to_csv(feature_data_output_path)
print(f"✅ Feature data saved to: {feature_data_output_path}")

✅ Feature data saved to: /workspaces/backtesting/investment-portfolio-project/data/feature_data.csv
